In [1]:
"""
Code to produce the results in section 5.2 of the paper 
Boltzmann Convolutions and Welford mean-variance layers with an application to 
time series forecasting and classification
"""

'\nCode to produce the results in section 5.2 of the paper \nBoltzmann Convolutions and Welford mean-variance layers with an application to \ntime series forecasting and classification\n'

In [2]:
#Import packages to facillitate this research 
import numpy as np 
import tensorflow as tf #(https://www.tensorflow.org/install)
from tensorflow.keras import (layers,Model) 
from tensorflow.keras.layers import (Layer, Input, Dense, Concatenate, Activation, TimeDistributed,
Flatten, Lambda, GlobalAveragePooling1D, LeakyReLU) 
from tensorflow.keras.utils import to_categorical 
from concretedropout.tensorflow import (ConcreteDenseDropout, ConcreteSpatialDropout1D, # https://github.com/aurelio-amerio/ConcreteDropout
get_weight_regularizer, get_dropout_regularizer) 
from sktime.classification.deep_learning import InceptionTimeClassifier #https://www.sktime.net/en/stable/get_started.html
from aeon.classification.convolution_based import MultiRocketHydraClassifier #https://www.aeon-toolkit.org/en/stable/installation.html
from aeon.classification.early_classification import TEASER
from aeon.classification.hybrid import HIVECOTEV2
from aeon.datasets import load_classification 
from sklearn.metrics import (accuracy_score, mean_absolute_error, mean_squared_error,  # https://scikit-learn.org/stable/install.html
classification_report)
from sklearn.model_selection import train_test_split
from math import comb
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping

/opt/anaconda3/lib/python3.12/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [3]:
file_path = '/Users/danielcoulson/Downloads/earnings_latest.csv'
df = pd.read_csv(file_path)
file_path = '/Users/danielcoulson/Downloads/stock_prices_latest.csv'
prices = pd.read_csv(file_path)
prices = prices.sort_values(by = [prices.columns[0], prices.columns[1]])
len(np.unique(prices.iloc[:,0]))
df = df.sample(frac = 1, random_state = 42).reset_index(drop = True)
df.iloc[:,1] = pd.to_datetime(df.iloc[:,1])
prices.iloc[:,1] = pd.to_datetime(prices.iloc[:,1])
time_series_list = []
before_earnings_list = []
on_and_after_earnings_list = []
ticker_prices_dict = {ticker:group for ticker, group in prices.groupby(prices.columns[0])}
valid_series_count = 0 
with tqdm(total = 168603   , desc = "Processing earnings dates") as pbar: 
    for index,row in df.iterrows(): 
        ticker = row[0]
        earnings_date = row[1]
        release_time = row.iloc[5]
        if pd.isna(release_time): 
            \
            continue 
        if release_time == "post": 
            earnings_date += pd.Timedelta(days = 1)
        elif release_time == "pre": 
            pass 
        else: 
            continue 
        if ticker in ticker_prices_dict: 
            ticker_prices = ticker_prices_dict[ticker]
            start_date_before = earnings_date - pd.Timedelta(days = 60 )
            before_earnings = ticker_prices[(ticker_prices.iloc[:,1] >= start_date_before) & (ticker_prices.iloc[:,1] < earnings_date)].iloc[:,6].values
            end_date_after = earnings_date + pd.Timedelta(days = 6)
            on_and_after_earnings = ticker_prices[(ticker_prices.iloc[:,1] >= earnings_date) & (ticker_prices.iloc[:,1] <= end_date_after)].iloc[:,6].values
            before_earnings_list.append(before_earnings)
            on_and_after_earnings_list.append(on_and_after_earnings)
            valid_series_count +=1
            pbar.update(1)
        if valid_series_count >= 168603 : 
            break
indices = [i for i in range(len(before_earnings_list)) if (len(before_earnings_list[i]) >= 40)]
indices1 = [i for i in range(len(on_and_after_earnings_list)) if len(on_and_after_earnings_list) != 0 ]
before_earnings_series = [before_earnings_list[i] for i in indices]
before_earnings_series = [series[-40:] for series in before_earnings_series]
on_and_after_earnings_series = [on_and_after_earnings_list[i] for i in indices]
on_earnings = []
for i in range(len(on_and_after_earnings_series)):
    if(len(on_and_after_earnings_series[i])>0):
        on_earnings.append(on_and_after_earnings_series[i][0])
before_earnings = np.array(before_earnings_series)
on_earnings = np.array(on_earnings)
before_earnings = np.expand_dims(before_earnings, axis = -1)
on_earnings = np.expand_dims(on_earnings, axis = -1) 
on_earnings = np.expand_dims(on_earnings, axis = -1) 
class_labels = []
for i in range(96614): 
    new = 0 
    new = on_earnings[i,0]
    old = 0 
    old = before_earnings[i,39,0]
    pc = 0 
    pc = ((new-old)/old) * 100 
    if pc>= 5 : 
        class_labels.append(1)
    else: 
        class_labels.append(0)
print(class_labels.count(0))
print(class_labels.count(1))

Processing earnings dates:  64%|████▍  | 107888/168603 [02:52<01:37, 623.97it/s]


53713
42901


In [4]:
class_labels = to_categorical(class_labels, num_classes = 2)
print(class_labels.shape)
X_observed_train = before_earnings[0:80000,:,:]
print(X_observed_train.shape)
X_observed_test = before_earnings[80000:90000,:,:]
print(X_observed_test.shape)
y_train = class_labels[0:80000,:]
print(y_train.shape)
y_test = class_labels[80000:90000,:]
print(y_test.shape)
X_forecast_train = on_earnings[0:80000,:,:]
print(X_forecast_train.shape)
X_forecast_test = on_earnings[80000:90000,:,:]
print(X_forecast_test.shape)

(96614, 2)
(80000, 40, 1)
(10000, 40, 1)
(80000, 2)
(10000, 2)
(80000, 1, 1)
(10000, 1, 1)


In [5]:
Ns = X_observed_train.shape[0]
time_steps = X_observed_train.shape[1]
n_forecast = X_forecast_train.shape[1]
n_classes = y_train.shape[1]
wr = get_weight_regularizer(Ns, l=1e-2, tau=0.5) #Determines the amount of regularization on the weights in the concrete dropout layers of ForeClassNet
dr = get_dropout_regularizer(Ns, tau=0.5, cross_entropy_loss=True) #Determines the amount of regularization on the dropout probabilities of ForeClassNet

In [6]:
def create_custom_filters(filter_length):
    """
    Function that takes in a user specified filter length to then construct hand-crafted filters 
    according to (8)-(10) in section 3.2 of the research paper. 
    """
    filter_1 = np.array([(-1) ** (i + 1) for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Decreasing filter
    filter_2 = np.array([(-1) ** i for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Increasing filter
    filter_3 = np.zeros(filter_length) #peaks filter 
    for n in range(0, int(((filter_length - 3) / 3) + 1)): #see section 3.2 for discussion of the formulas
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), n)
    for n in range(int(filter_length / 3), int((2 * (filter_length - 3) / 3) + 2)):
        filter_3[n] = (6 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (filter_length / 3)))
    for n in range(int((2 * (filter_length - 3) / 3) + 2), filter_length):
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (2 * (filter_length - 3) / 3) - 2))
    filter_3 = filter_3.reshape(filter_length, 1, 1)
    custom_filters = np.concatenate([filter_1, filter_2, filter_3], axis=2) #combine the filters together
    return custom_filters
class WelfordMeanVarianceLayer(Layer): #Defining the Welford Mean-variance layer (section 2.2) as a subclass of Layer in Keras 
    def __init__(self, total_series, n_forecast=4, **kwargs): #constructor of the class, where it takes in the number of time series we want to have mean and variance estimates for and the number of forecasts for each 
        super(WelfordMeanVarianceLayer, self).__init__(**kwargs) #calls the Keras layer class
        self.total_series = total_series #number of series the layer will track 
        self.n_forecast = n_forecast #number of forecast steps ahead 
        self.stats = self.add_weight( #creates a non-trainable tensor to store the means and variances which is initialized with zeros
            shape=(self.total_series, self.n_forecast, 2),
            initializer="zeros",
            trainable=False,
            name="stats"
        ) 
        self.count = self.add_weight( #creates a non-trainable tensor to count how many samples for the mean and variance we have for the forecasts of a given time series, which are obtained through successive forward passes
            shape=(self.total_series,),
            initializer="zeros",
            trainable=False,
            name="count"
        )
    def reset_statistics(self): #A method to clear the existing estimates of the mean and variance of the forecasts and set them to zero, which you might want to do during testing
        self.stats.assign(tf.zeros_like(self.stats))
        self.count.assign(tf.zeros_like(self.count))
    def call(self, inputs, **kwargs): #main method used when calling the Welford layer in a given forward pass
        forecast_values, series_ids = inputs  #unpacks the inputs into the Welford layer, the forecast values in the current forward pass and the ids of the current time series being processed. 
        batch_size = tf.shape(forecast_values)[0] #derives the current batch size from the shape of the forecast values tensor
        def loop_body(i, current_stats, current_count): #defines the body of a while loop to be used in the proceeding lines. Processes the ith time series in the batch by applying applying Welford's algorithm 
            idx = series_ids[i] #determines the id of the current time series being processed        
            val = forecast_values[i] #the values to be used to update the mean and variances of the forecasts
            current_count = tf.tensor_scatter_nd_add(current_count, [[idx]], [1.0]) #increments the count of the number of times we have updated the mean and variance of the given time series 
            old_mean = current_stats[idx, :, 0] #extracts the old forecast means 
            old_var  = current_stats[idx, :, 1] #extracts the old forecast variance 
            n = current_count[idx] #number of times we have updated the mean and variance of the forecasts for a given time series 
            delta = val - old_mean #the difference between the new forecast value and the old mean 
            new_mean = old_mean + (delta /n) #formula used in Welford's algorithm to update the mean 
            delta2 = val - new_mean #difference between the new forecast value and the new mean 
            new_var = ((1-(1/n))*old_var) + ((delta*delta2)/n) #update the value of the sample variance using the population variance formula      
            updated_stats = tf.stack([new_mean, new_var], axis=-1) #combines the new values of the sample mean and variance 
            current_stats = tf.tensor_scatter_nd_update(current_stats, [[idx]], [updated_stats]) #overwrites the old values of the mean and variance for a given time series(idx) with the new values of the mean and variance 
            return i+1, current_stats, current_count #set i to i+1 to go to the next time series in the batch, and return the current values of the sample mean and variance, and the number of times the time series have been processed
        i0 = tf.constant(0) #starting index of the loop 
        stats0 = self.stats #the initial stats for the start of the while loop 
        count0 = self.count #the initial counts that each time series has been processed for the start of the while loop 
        _, updated_stats, updated_count = tf.while_loop( #the while loop which outputs the index, the updated stats, and the updated counts 
            cond=lambda i, *_: i < batch_size, #determines if the while loop should continue or not, where the loop stops if i exceeds the batch size as there are no more time series to process in the current batch 
            body=loop_body, #using the function defined earlier which is used in each iteration of the while loop 
            loop_vars=[i0, stats0, count0] #initial values of the variables in the while loop 
        )
        self.stats.assign(updated_stats) #permanently updates the stats variable of the layer which will be used in future calls of the layer 
        self.count.assign(updated_count) #permanently updates the count variable for the layer which will be used in future calls of the layer 
        out_stats_for_batch = tf.gather(self.stats, series_ids) #takes the means and variances for the forecasted time points for the time series in the current batch so they can be passed to the proceeding layer 
        return out_stats_for_batch #returns the means and variances for the forecasted time series in the current batch so they can be passed into the next layer of the network in the current forward pass 
class BoltzmannConv1D(Layer): #Defining the Boltzmann convolutional layer (section 2.1) as a subclass of Layer in Keras 
    def __init__(self, out_channels, kernel_sizes, dilation_rate=1, padding="causal", temperature=1, **kwargs):#contructor of the class which takes in several hyperparameters including the type of padding, dilation rate, and temperature of the Boltzmann distribution
        super(BoltzmannConv1D, self).__init__(**kwargs) #Calls the Keras layer class 
        self.out_channels = out_channels #number of output channels 
        self.kernel_sizes = kernel_sizes #the set of kernel filter lengths 
        self.dilation_rate = dilation_rate #dilation rate of the layer 
        self.padding = padding #type of padding for the layer 
        self.temperature = temperature #value of the temperature for the Boltzmann distribution 
        self.convs = [  #creates a list consisting of sub-convolutional layers with the parameters defined above. The layer is wrapped in a Spatial Concrete Dropout layer 
            ConcreteSpatialDropout1D(
                tf.keras.layers.Conv1D(
                    out_channels, ks,
                    dilation_rate=dilation_rate,
                    padding=padding
                ),
                is_mc_dropout=True,
                weight_regularizer=wr,
                dropout_regularizer=dr
            )
            for ks in kernel_sizes #iterates over the filter lengths in the kernel_sizes list creating associated convolutional layers
        ]
    def build(self, input_shape): #create the weights used in the layers 
        self.probs = self.add_weight(   #logits used in the Boltzmann convolution distribution  which are learnable 
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(BoltzmannConv1D, self).build(input_shape) #finalizes the build method by inheriting the build from the overall Keras layer class 
    def call(self, x, training=False): #main method used when calling the Boltzmann layer for a given forward pass 
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature) #defines the probability discussed in section 2.1 
        conv_outputs = [ #creates the list of the weighted convolution outputs 
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities)) #turns the kernel probabilities into a list which are then paired with their associated convolutional layer, with them being multiplied together 
        ]
        out = tf.add_n(conv_outputs) #sums up the weighted convolutional outputs in the list 
        return out #returns the summed output of the weighted convolutional layers according to (2) in section 2.1 
class PredefinedBoltzmannConv1D(Layer):
    def __init__(self, filters, kernel_sizes, dilation_rate, custom_filters, temperature=1, **kwargs):
        super(PredefinedBoltzmannConv1D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_sizes = kernel_sizes
        self.temperature = temperature
        self.custom_filters = custom_filters #pre-defined filters as described in (8)-(10) in section 3.2
        self.dilation_rate = dilation_rate
        self.convs = []
        for ks in kernel_sizes:
            conv_layer = tf.keras.layers.Conv1D(
                filters=filters,
                kernel_size=ks,
                padding="causal",
                use_bias=False,
                dilation_rate=dilation_rate
            )
            self.convs.append(conv_layer)
    def build(self, input_shape):
        for i, (conv_layer, ks) in enumerate(zip(self.convs, self.kernel_sizes)):
            conv_layer.build(input_shape)
            filter_weights = self.custom_filters[ks]
            conv_layer.set_weights([filter_weights])
            conv_layer.trainable = False
        self.probs = self.add_weight(
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(PredefinedBoltzmannConv1D, self).build(input_shape)
    def call(self, x, training=False):
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature)
        conv_outputs = [
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities))
        ]
        out = tf.add_n(conv_outputs)
        return out

In [7]:
kernel_sizes = [3,6,12,18,24,30,36] #pre-defined filter lengths (section 2.1)
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes} #pre-defined filter weights as a function of filter length (8)-(10) in section 3.2
ts_input = Input(shape=(time_steps, 1), name="ts_input")   #defining the input time series to ForeClassNet       
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")   #defining the input time series index to ForeClass Net 

#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connectedlayer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 90000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])
concatenated_2 = Concatenate(axis=1)([x6_final, output_forecast_final])

#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8)(concatenated_2)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#compilation and summary of ForeClassNet#
model = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 40, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 40, 3)     │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 40, 128)   │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 40, 3)     │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 40, 128)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 40, 131)   │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 40, 64)    │  1,081,998 │ concatenate[0][0] │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 40, 64)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 40, 32)    │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 40, 32)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 40, 1)     │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 40)        │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      1,313 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 40, 1)     │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 1)         │         34 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 40, 1)     │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -               

 Total params: 1,644,183 (6.27 MB)

 Trainable params: 1,373,796 (5.24 MB)

 Non-trainable params: 270,387 (1.03 MB)

In [8]:
early_stopping = EarlyStopping(monitor='val_activation_accuracy', patience=10,restore_best_weights=True,verbose=1, mode='max')
series_id_train = np.arange(Ns, dtype=np.int32)
batch_size = 128
epochs = 100
tic = time.perf_counter()
history = model.fit( x = [X_observed_train,series_id_train],y = [X_forecast_train,y_train],epochs=epochs,batch_size=batch_size,
                    validation_split = 0.1,callbacks=[early_stopping], shuffle = True)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 141s 232ms/step - activation_accuracy: 0.5852 - activation_loss: 0.6325 - loss: 214194487296.0000 - output_forecast_loss: 214193815552.0000 - output_forecast_mse: 214194487296.0000 - val_activation_accuracy: 0.6672 - val_activation_loss: 0.6078 - val_loss: 621794754560.0000 - val_output_forecast_loss: 616859893760.0000 - val_output_forecast_mse: 621794754560.0000
Epoch 2/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 132s 235ms/step - activation_accuracy: 0.6765 - activation_loss: 1.3791 - loss: 329355788288.0000 - output_forecast_loss: 329355165696.0000 - output_forecast_mse: 329355788288.0000 - val_activation_accuracy: 0.6360 - val_activation_loss: 0.6410 - val_loss: 621481426944.0000 - val_output_forecast_loss: 616549056512.0000 - val_output_forecast_mse: 621481426944.0000
Epoch 3/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 131s 234ms/step - activation_accuracy: 0.6746 - activation_loss: 0.6256 - loss: 187611545600.0000 - output_forecast_loss: 187610890240.0000 - out

In [9]:
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #reset the statistics in the Welford layer

In [10]:
def mc_dropout_predict(model, X_test, test_ids, num_samples=100):
    """
    Peforms Monte Carlo dropout, where dropout is applied at inference time. 
    This takes as input the trained ForeClassNet model, the input observed time series, 
    the id's associated with the time series in the test set, and the number of MC samples which
    is the number of forward passes through the trained ForeClassNet model. 
    """
    forecast_predictions = [] #list to store the forecast output from each forward pass through ForeClassNet
    classification_predictions = [] #List to store the classification output from each forwards pass through ForeClassNet
    for _ in tqdm(range(num_samples), desc="MC Dropout passes"):
        forecast, classification = model.predict([X_test, test_ids], verbose=0) #performs a forward pass through the network 
        forecast_predictions.append(forecast) #add the forecats array to the list 
        classification_predictions.append(classification) #add the classification array of probabilities to the list
    forecast_predictions = np.array(forecast_predictions)
    classification_predictions = np.array(classification_predictions) 
    mean_forecast = np.mean(forecast_predictions, axis=0) #take the mean with respect to the MC samples of the forecast output
    std_forecast = np.std(forecast_predictions, axis=0) #take the standard deviation with respect to the MC samples of the forecast output
    mean_classification = np.mean(classification_predictions, axis=0) #take the mean with respect to the MC samples of the classification probabilities
    std_classification = np.std(classification_predictions, axis=0) #take the standard deviation with respect to the MC samples of the classification probabilities
    return (mean_forecast, std_forecast), (mean_classification, std_classification)

In [11]:
# Perform MC Dropout predictions with your test data and test IDs
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,X_observed_test,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [16:58<00:00, 10.19s/it]

Predicted classes: [0 0 1 ... 0 1 1]
0.7502
              precision    recall  f1-score   support

           0       0.77      0.74      0.75      5100
           1       0.74      0.77      0.75      4900

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



In [12]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_observed_train,y_train,test_size=0.1,random_state=42)
patience = 10;best_val_acc = 0.0;no_improve_count = 0;best_model = None;best_epoch_count = 0;max_epochs_to_try = epochs
tic = time.perf_counter()
for epoch_count in range(1, max_epochs_to_try + 1):
    print(f"\nTraining a fresh model for {epoch_count} epoch(s)...")
    clf = InceptionTimeClassifier(n_epochs=epoch_count,batch_size=128)
    clf.fit( X_train_sub.transpose(0, 2, 1),np.argmax(y_train_sub, axis=1))
    val_preds = clf.predict(X_val.transpose(0, 2, 1))
    val_acc = accuracy_score(np.argmax(y_val, axis=1), val_preds)
    print(f"Validation Accuracy after {epoch_count} epoch(s): {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = clf
        best_epoch_count = epoch_count
        no_improve_count = 0  
        print("New best model")
    else:
        no_improve_count += 1
        print(f"No improvement. {no_improve_count} consecutive runs without improvement.")
    if no_improve_count >= patience:
        print(f"\nNo improvement for {patience} consecutive runs. Stopping early.")
        break
    toc = time.perf_counter()
print(f"\nBest validation accuracy: {best_val_acc:.4f} (achieved at {best_epoch_count} epoch(s))")
if best_model is not None:
    predictions = best_model.predict(X_observed_test.transpose(0, 2, 1))
    classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
    print(f"Classification Test Accuracy: {classification_accuracy:.4f}")
    classification_report_str = classification_report(
        np.argmax(y_test, axis=1),
        predictions
    )
    print("Classification Report on Test Set:\n", classification_report_str)
else:
    print("No best model was found (check if training never improved).")


Training a fresh model for 1 epoch(s)...
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
Validation Accuracy after 1 epoch(s): 0.5634
New best model

Training a fresh model for 2 epoch(s)...
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
Validation Accuracy after 2 epoch(s): 0.5634
No improvement. 1 consecutive runs without improvement.

Training a fresh model for 3 epoch(s)...
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
Validation Accuracy after 3 epoch(s): 0.5634
No improvement. 2 consecutive runs without improvement.

Training a fresh model for 4 epoch(s)...
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Validation Accuracy after 4 epoch(s): 0.5634
No improvement. 3 consecutive runs without improvement.

Training a fresh model for 5 epoch(s)...
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
Validation Accuracy after 5 epoch(s): 0.5634
No improvement. 4 consecutive runs without improvement.

Training a fresh model for 6 epoch(s)...
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
Validation Accuracy after 6 epoch(s): 0.5634

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
X_observed_train_subset = before_earnings[0:10000,:,:]
y_train_subset = class_labels[0:10000,:]
print(X_observed_train_subset.shape)
print(y_train_subset.shape)

(10000, 40, 1)
(10000, 2)


In [14]:
train_class_labels = np.argmax(y_train, axis=1)
class_0_indices = np.where(train_class_labels == 0)[0]
class_1_indices = np.where(train_class_labels == 1)[0]
class_0_indices_subset = class_0_indices[:5000]
class_1_indices_subset = class_1_indices[:5000]
balanced_indices = np.concatenate([class_0_indices_subset, class_1_indices_subset])
np.random.shuffle(balanced_indices)
X_observed_train_subset = X_observed_train[balanced_indices]
y_train_subset = y_train[balanced_indices]
print("Subset shapes:")
print("X_observed_train_subset:", X_observed_train_subset.shape)
print("y_train_subset:", y_train_subset.shape)

Subset shapes:
X_observed_train_subset: (10000, 40, 1)
y_train_subset: (10000, 2)


In [15]:
MRhydra = MultiRocketHydraClassifier() #initialize the MR-Hydra classifier 
MRhydra.fit(X_observed_train_subset.transpose(0, 2, 1), np.argmax(y_train_subset,axis = 1))  #train the MR-Hydra classifier on the observed time series 
predictions = MRhydra.predict(X_observed_test.transpose(0, 2, 1)) #test the trained MR-Hydra classifier on the observed test set 
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

0.5471
              precision    recall  f1-score   support

           0       0.56      0.56      0.56      5100
           1       0.54      0.53      0.53      4900

    accuracy                           0.55     10000
   macro avg       0.55      0.55      0.55     10000
weighted avg       0.55      0.55      0.55     10000



In [16]:
early = TEASER(classification_points=[40]) #initalize the teaser classifier, specifying we want to make the classification decision after we have observed 40 time points 
tic = time.perf_counter()
early.fit(X_observed_train_subset.transpose(0,2,1), np.argmax(y_train_subset,axis = 1)) #train the teaser classifier 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
predictions, decisions = early.predict(X_observed_test.transpose(0,2,1)) #evaluate the trained teaser classifier 
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Time taken for fitting: 201.69 seconds
Classification Test Accuracy: 0.5251
              precision    recall  f1-score   support

           0       0.53      0.54      0.54      5100
           1       0.52      0.51      0.51      4900

    accuracy                           0.53     10000
   macro avg       0.52      0.52      0.52     10000
weighted avg       0.52      0.53      0.52     10000



In [17]:
hc2 = HIVECOTEV2(time_limit_in_minutes= 31)
y_train_labels = np.argmax(y_train_subset, axis=1)
tic = time.perf_counter()
hc2.fit(X_observed_train_subset.transpose(0, 2, 1),y_train_labels)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
predictions = hc2.predict(X_observed_test.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
from sklearn.metrics import classification_report
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

Time taken for fitting: 4526.12 seconds
Classification Test Accuracy: 0.5767
              precision    recall  f1-score   support

           0       0.58      0.62      0.60      5100
           1       0.57      0.53      0.55      4900

    accuracy                           0.58     10000
   macro avg       0.58      0.58      0.58     10000
weighted avg       0.58      0.58      0.58     10000



In [18]:
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #reset the statistics in the Welford layer
model = 0 
tf.keras.backend.clear_session()

In [19]:
Ns = X_observed_train_subset.shape[0]
time_steps = X_observed_train_subset.shape[1]
X_forecast_train_subset = X_forecast_train[balanced_indices]
n_forecast = X_forecast_train_subset.shape[1]
n_classes = y_train_subset.shape[1]
wr = get_weight_regularizer(Ns, l=1e-2, tau=0.5) #Determines the amount of regularization on the weights in the concrete dropout layers of ForeClassNet
dr = get_dropout_regularizer(Ns, tau=0.5, cross_entropy_loss=True) #Determines the amount of regularization on the dropout probabilities of ForeClassNet

In [20]:
kernel_sizes = [3,6,12,18,24,30,36] #pre-defined filter lengths (section 2.1)
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes} #pre-defined filter weights as a function of filter length (8)-(10) in section 3.2
ts_input = Input(shape=(time_steps, 1), name="ts_input")   #defining the input time series to ForeClassNet       
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")   #defining the input time series index to ForeClass Net 

#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connectedlayer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 20000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])
concatenated_2 = Concatenate(axis=1)([x6_final, output_forecast_final])

#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8)(concatenated_2)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#compilation and summary of ForeClassNet#
model = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 40, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 40, 3)     │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 40, 128)   │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 40, 3)     │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 40, 128)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 40, 131)   │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 40, 64)    │  1,081,998 │ concatenate[0][0] │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 40, 64)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 40, 32)    │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 40, 32)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 40, 1)     │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 40)        │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      1,313 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 40, 1)     │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 1)         │         34 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 40, 1)     │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -               

 Total params: 1,434,183 (5.47 MB)

 Trainable params: 1,373,796 (5.24 MB)

 Non-trainable params: 60,387 (235.89 KB)

In [21]:
early_stopping = EarlyStopping(monitor='val_activation_accuracy', patience=10,restore_best_weights=True,verbose=1, mode='max')
series_id_train = np.arange(Ns, dtype=np.int32)
batch_size = 128
epochs = 100
tic = time.perf_counter()
history = model.fit( x = [X_observed_train_subset,series_id_train],y = [X_forecast_train_subset,y_train_subset],epochs=epochs,batch_size=batch_size,
                    validation_split = 0.1,callbacks=[early_stopping], shuffle = True)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 27s 226ms/step - activation_accuracy: 0.5397 - activation_loss: 3.4495 - loss: 378539474944.0000 - output_forecast_loss: 378466074624.0000 - output_forecast_mse: 378539474944.0000 - val_activation_accuracy: 0.5100 - val_activation_loss: 0.7894 - val_loss: 14725251.0000 - val_output_forecast_loss: 14380198.0000 - val_output_forecast_mse: 14725250.0000
Epoch 2/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 15s 206ms/step - activation_accuracy: 0.5482 - activation_loss: 31.0636 - loss: 318168563712.0000 - output_forecast_loss: 318091853824.0000 - output_forecast_mse: 318168563712.0000 - val_activation_accuracy: 0.5700 - val_activation_loss: 10.3256 - val_loss: 14756047.0000 - val_output_forecast_loss: 14410276.0000 - val_output_forecast_mse: 14756038.0000
Epoch 3/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 15s 212ms/step - activation_accuracy: 0.5614 - activation_loss: 0.9022 - loss: 115071762432.0000 - output_forecast_loss: 119623892992.0000 - output_forecast_mse: 115071762432.

In [22]:
# Perform MC Dropout predictions with your test data and test IDs
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,X_observed_test,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [12:55<00:00,  7.76s/it]

Predicted classes: [0 0 1 ... 0 1 1]
0.7304
              precision    recall  f1-score   support

           0       0.76      0.69      0.72      5100
           1       0.71      0.77      0.74      4900

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000



In [23]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_observed_train_subset,y_train_subset,test_size=0.1,random_state=42)
patience = 10;best_val_acc = 0.0;no_improve_count = 0;best_model = None;best_epoch_count = 0;max_epochs_to_try = epochs
tic = time.perf_counter()
for epoch_count in range(1, max_epochs_to_try + 1):
    print(f"\nTraining a fresh model for {epoch_count} epoch(s)...")
    clf = InceptionTimeClassifier(n_epochs=epoch_count,batch_size=128)
    clf.fit( X_train_sub.transpose(0, 2, 1),np.argmax(y_train_sub, axis=1))
    val_preds = clf.predict(X_val.transpose(0, 2, 1))
    val_acc = accuracy_score(np.argmax(y_val, axis=1), val_preds)
    print(f"Validation Accuracy after {epoch_count} epoch(s): {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = clf
        best_epoch_count = epoch_count
        no_improve_count = 0  
        print("New best model")
    else:
        no_improve_count += 1
        print(f"No improvement. {no_improve_count} consecutive runs without improvement.")
    if no_improve_count >= patience:
        print(f"\nNo improvement for {patience} consecutive runs. Stopping early.")
        break
    toc = time.perf_counter()
print(f"\nBest validation accuracy: {best_val_acc:.4f} (achieved at {best_epoch_count} epoch(s))")
if best_model is not None:
    predictions = best_model.predict(X_observed_test.transpose(0, 2, 1))
    classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
    print(f"Classification Test Accuracy: {classification_accuracy:.4f}")
    classification_report_str = classification_report(
        np.argmax(y_test, axis=1),
        predictions
    )
    print("Classification Report on Test Set:\n", classification_report_str)
else:
    print("No best model was found (check if training never improved).")


Training a fresh model for 1 epoch(s)...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Validation Accuracy after 1 epoch(s): 0.5000
New best model

Training a fresh model for 2 epoch(s)...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Validation Accuracy after 2 epoch(s): 0.5000
No improvement. 1 consecutive runs without improvement.

Training a fresh model for 3 epoch(s)...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Validation Accuracy after 3 epoch(s): 0.5010
New best model

Training a fresh model for 4 epoch(s)...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Validation Accuracy after 4 epoch(s): 0.5010
No improvement. 1 consecutive runs without improvement.

Training a fresh model for 5 epoch(s)...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Validation Accuracy after 5 epoch(s): 0.5010
No improvement. 2 consecutive runs without improvement.

Training a fresh model for 6 epoch(s)...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Validation Accuracy after 6 epoch(s): 0.5010
No improvement. 3 consecutive runs without improveme

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
